# Using super and the MRO to make things work for you.

I've defined Base and SuperBase in `helpers.py`, so they can be imported other classes we will redefine. 

Base for our purpose is just `object` that prints its own name. We expect to see it so we know when the very bottom of certain paths gets hit.

## Multiple inheritance with multiple arguments

The previous examples didn't give arguments to their `__init__` lets correct that.

In [2]:
from helpers import Base, SuperBase

In [4]:
class ChildA(SuperBase):
    def __init__(self, A):
        print('Enter ChildA.__init__')
        self.a = A
        super().__init__()
        print('Exit ChildA.__init__')
    
    def my_arg(self):
        print(f'ChildA.my_arg: {self.a}')

class ChildB(ChildA):
    def __init__(self, A, B):
        print('Enter ChildB.__init__')
        self.b = B
        super().__init__(A)
        print('Exit ChildB.__init__')

    def my_arg(self):
        print(f'ChildB.my_arg: {self.b}')

my_class = ChildB(1, 2)
my_class.my_arg()

Enter ChildB.__init__
Enter ChildA.__init__
Enter SuperBase.__init__
Exit SuperBase.__init__
Exit ChildA.__init__
Exit ChildB.__init__
ChildB.my_arg: 2


During that process which we could have continued 'turtles all the way down' we made ChildB inherit ChildA. Child B has attributes a and b, but the method my_arg uses only that defined by ChildB. Enter super.

In [6]:
class ChildC(ChildA):
    def __init__(self, A, C):
        print('Enter ChildC.__init__')
        self.c = C
        super().__init__(A)
        print('Exit ChildC.__init__')

    def my_arg(self):
        print(f'ChildC.my_arg: {self.c}')
        super().my_arg()

my_class = ChildC(1, 3)
my_class.my_arg()

Enter ChildC.__init__
Enter ChildA.__init__
Enter SuperBase.__init__
Exit SuperBase.__init__
Exit ChildA.__init__
Exit ChildC.__init__
ChildC.my_arg: 3
ChildA.my_arg: 1


The ability to run that code was not lost just tucked away in a super, a super just looks for the next definition of the requested function name in the MRO.

In fact in the `__init__` method the number of arguments is being whittled away from 2 to 1 but you can do this quite creatively a complex class with lots of `__init__` arguments may quite quickly split them up into multiple inherited `__init__`.

In [7]:
# The super looks for the next definition of the method in the MRO
# We shouldn't but could get weird with that.

class ChildD(ChildA):
    def __init__(self, A, D):
        print('Enter ChildD.__init__')
        self.d = D
        self._a = A
        print('Exit ChildD.__init__')

    def my_arg(self):
        print(f'ChildD.my_arg: {self.d}')
        super().__init__(self._a)
        super().my_arg()

my_class = ChildD(1, 4)
my_class.my_arg()

Enter ChildD.__init__
Exit ChildD.__init__
ChildD.my_arg: 4
Enter ChildA.__init__
Enter SuperBase.__init__
Exit SuperBase.__init__
Exit ChildA.__init__
ChildA.my_arg: 1


### That was grim, and confusing.

Let's make it worse!

In [13]:
class ChildE(ChildA):
    def __init__(self, A, E):
        print('Enter ChildE.__init__')
        self.e = E
        super().__init__(A)
        print('Exit ChildE.__init__')

class ChildF(ChildE):
    def __init__(self, A, E, F):
        print('Enter ChildF.__init__')
        self.f = F
        super().__init__(A, E)
        print('Exit ChildF.__init__')
    
    def my_arg(self):
        print(f'ChildF.my_arg: {self.f}')
        super().my_arg()

my_class = ChildF(1, 5, 6)
my_class.my_arg()

print(f"ChildF MRO:{ChildF.__mro__}")

Enter ChildF.__init__
Enter ChildE.__init__
Enter ChildA.__init__
Enter SuperBase.__init__
Exit SuperBase.__init__
Exit ChildA.__init__
Exit ChildE.__init__
Exit ChildF.__init__
ChildF.my_arg: 6
ChildA.my_arg: 1
ChildF MRO:(<class '__main__.ChildF'>, <class '__main__.ChildE'>, <class '__main__.ChildA'>, <class 'helpers.SuperBase'>, <class 'object'>)


## Wait, what?

`super` just jumped child E and went to A I thought it was accessing the parent!?

No super searches MRO in order until it finds that method. If you are familiar with `PATH` resolution then you can intuit this the same way. If you are unfamiliar with `PATH` resolution you just learnt two things!

# Get back to multiple inheritance please.

The next step is likely figuring out how the correct input variables make it to the correct `__init__` methods.

Starting with a design lets say we want this:

% TODO: Make Figure